In [26]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [27]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.03

In [29]:
# MNIST dataset
import torch.utils
import torch.utils.data


train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [30]:
# Bidirectional recurrent neural network (many-to-one)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)   # 2 for bidirection
        
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for birection
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [31]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 1.9283
Epoch [1/2], Step [200/600], Loss: 1.5292
Epoch [1/2], Step [300/600], Loss: 1.3991
Epoch [1/2], Step [400/600], Loss: 0.9178
Epoch [1/2], Step [500/600], Loss: 0.8784
Epoch [1/2], Step [600/600], Loss: 0.8581
Epoch [2/2], Step [100/600], Loss: 0.7359
Epoch [2/2], Step [200/600], Loss: 0.6698
Epoch [2/2], Step [300/600], Loss: 0.5517
Epoch [2/2], Step [400/600], Loss: 0.5041
Epoch [2/2], Step [500/600], Loss: 0.3919
Epoch [2/2], Step [600/600], Loss: 0.6021


In [33]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Test Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Test Accuracy of the model on the 10000 test images: 85.49 %


In [35]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')